# Práctica 2 - Parte 2: Sistema masa-resorte con rozamiento

En esta práctica se estudiará el sistema masa-resorte-amortiguador clásico, que es un ejemplo típico de sistema dinámico de segundo orden.

---

## Enunciado

1. Considera el sistema masa-resorte-amortiguador descrito por la ecuación diferencial:

   $
   m \ddot{x} + b \dot{x} + k x = 0
   $
  
   donde:
   - $m$ es la masa,
   - $b$ es el coeficiente de rozamiento,
   - $k$ es la constante del resorte.

2. Implementa el modelo en Python y simula la evolución temporal para distintos valores de $b$, con $m$ y $k$ fijos.

3. Ajusta los parámetros para obtener los distintos tipos de comportamiento dinámico:
   - Sin amortiguamiento ($\zeta=0$).
   - Subamortiguado ($0 < \zeta < 1$).
   - Críticamente amortiguado ($\zeta=1$).
   - Sobreamortiguado ($\zeta>1$).

4. Representa gráficamente las respuestas y compara los resultados.

---

## Ejercicio propuesto

Implementa en Python el sistema masa resorte con rozamiento y representa la evolución temporal $x(t)$ con distintos valores de los parámetros. Ajusta los parámetros para obtener los comportamientos que se indican en el punto 3.

---

## Ejercicio propuesto OPCIONAL

Representa las raíces en el plano complejo para cada uno de los casos.

## Preguntas de reflexión

1. ¿Qué diferencias observas en la respuesta entre el caso sin amortiguamiento y el caso subamortiguado?
2. ¿Por qué en el caso crítico la respuesta es la más rápida posible sin oscilaciones?
3. ¿Qué efecto tiene un rozamiento excesivo (sobreamortiguado) sobre la rapidez de la respuesta?
4. ¿Cómo se relaciona el coeficiente de amortiguamiento $\zeta$ con la posición de las raíces en el plano complejo?
